In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install() }
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


### Nasa Mars News

In [47]:
# Visit the URL of page to be scraped
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)

# Parse Results HTML with BeautifulSoup
nasa_html = browser.html
soup_nasa = BeautifulSoup(nasa_html, "html.parser")

#.find returns the first item. .find_all returns all items (here we need the first one only)
#required_list = soup_nasa.select_one("ul.item_list li.slide")----(another method)
required_list = soup_nasa.find('ul', class_='item_list')

required_list.find("div", class_="content_title")

# Scrape the Latest News Title
# .text or .get_text()
title = required_list.find("div", class_="content_title").get_text()
print(title)

# Scrape the Latest Paragraph Text
title_paragraph = required_list.find("div", class_="article_teaser_body").get_text()
print(title_paragraph)

NASA to Host Virtual Briefing on February Perseverance Mars Rover Landing
NASA leadership and members of the mission will discuss the agency’s latest rover, which touches down on the Red Planet on Feb. 18.


### JPL, NASA - Featured Image

In [83]:
url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_img)

jpl_html=browser.html
jpl_soup = BeautifulSoup(jpl_html,'html.parser')

# Navigate to navbar and click on 4th button 'Galleries'
first_button = browser.find_by_tag('button')
first_button[4].click()

#Extract the title of the image
img_title_soup = BeautifulSoup(browser.html,'html.parser')
title = img_title_soup.find('div', class_='col-span-3').text
image_title = title.split('.')
img_title = image_title[1]
print(img_title)

#Click on 'Featured Image' text (link)
browser.links.find_by_partial_text('Featured Image').click()

 Supercam's Mars Meteorite Aboard the ISS


In [75]:
# img_soup = BeautifulSoup(browser.html,'html.parser')
# img_soup2 = img_soup.find('div', class_='PageImageDetail ThemeLight')
# img_title = img_soup2.find('h1', class_='text-h2').text
# img_title

In [84]:
# #Click on 'Download JPG' button to get full size image
browser.links.find_by_partial_text('Download JPG').click()

# #Scrape the webpage to get the image url
featured_image_soup = BeautifulSoup(browser.html,'html.parser')
featured_url = featured_image_soup.find_all('img')[0]['src']
featured_url

'https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24179.jpg'

### Mars facts table

In [52]:
# Visit the Mars Facts webpage 
mars_url = 'https://space-facts.com/mars/'
browser.visit(mars_url)

# Parse Results HTML with BeautifulSoup
mars_html = browser.html
soup_mars = BeautifulSoup(mars_html, "html.parser")

In [53]:
tables = pd.read_html(mars_url)
len(tables)

3

In [54]:
facts_table = tables[0]
facts_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [55]:
facts_table.columns=['Description', 'Value']
facts_table

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### Hemispheres of Mars

In [3]:
# Visit the USGS Astrogeology site
url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

# Parse HTML with Beautiful Soup
hemi_soup = BeautifulSoup(browser.html,'html.parser')

# Store the main url 
main_url = "https://astrogeology.usgs.gov"

# Create empty list for hemisphere urls and titles
hemisphere_image_urls = []

# Retreive all items that contain mars hemispheres information
url_list = hemi_soup.find_all('div', class_='description')

# Loop through the items previously stored
for item in url_list:
    title = item.find('h3').text
    #print(title)
    
    # Store link that leads to full image website
    partial_url = item.find('a')['href']
    #print(partial_url)
    
    #Create the complete url to the full image page
    partial_url2 = main_url+partial_url
    #print(hemi_url)
    
    # Visit the link that contains the full image website 
    browser.visit(partial_url2)
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    url_soup =BeautifulSoup(browser.html, 'html.parser')
    
    hemi_find = url_soup.find_all('div', class_='downloads')
    
    # Retrieve full image source 
    for x in hemi_find:
        final_url = x.find('a', target="_blank")['href']
        #print(final_url)
        
        # Append the retreived information into a list of dictionaries 
        hemisphere_image_urls.append({"title": title, "hemi_url": final_url})
        
# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [ ]:
browser.quit()